In [ ]:
!pip install huggingface-hub transformers -q

In [ ]:
from transformers import pipeline
from langchain.schema.runnable import RunnableSequence, RunnableMap, RunnableLambda
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from huggingface_hub import login

# import os
# from dotenv import load_dotenv
# load_dotenv()


In [ ]:
login("")

In [ ]:
# --- Step 1: Initialize Hugging Face Pipelines and Gemini LLM ---
translation_hf_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
sentiment_hf_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7, api_key="")

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# --- Step 2: Define Prompts ---
response_prompt = PromptTemplate(
    input_variables=["translated_feedback", "sentiment"],
    template=(
        "Based on the following customer feedback and sentiment, write a polite and helpful 20-word response to be sent to the customer:\n\n"
        "Feedback: {translated_feedback}\n"
        "Sentiment: {sentiment}\n\n"
        "Response: "
    )
)

In [ ]:
# --- Step 3: Create Runnable Sequence ---
def translate(inputs):
    print('\n')
    print(inputs)
    return {"translated_feedback": translation_hf_pipeline(inputs["feedback"])[0]["translation_text"]}

def analyze_sentiment(inputs):
    return {**inputs, "sentiment": sentiment_hf_pipeline(inputs["translated_feedback"])[0]["label"]}

def generate_response(inputs):
    print('\n')
    print(inputs)
    prompt_text = response_prompt.format(
        translated_feedback=inputs["translated_feedback"],
        sentiment=inputs["sentiment"]
    )
    # Use llm.invoke() to get the response content.
    response = llm.invoke(prompt_text).content
    return {"response": response}

runnable_sequence = RunnableSequence(
    RunnableLambda(translate),
    RunnableLambda(analyze_sentiment),
    RunnableLambda(generate_response)
)


In [ ]:
# --- Step 4: Run Pipeline ---
customer_feedback = "Esta es la mejor experiencia de compra que he tenido, pero el envío fue un poco lento."
result = runnable_sequence.invoke({"feedback": customer_feedback})



{'feedback': 'Esta es la mejor experiencia de compra que he tenido, pero el envío fue un poco lento.'}


{'translated_feedback': "This is the best shopping experience I've ever had, but shipping was a little slow.", 'sentiment': '4 stars'}


In [ ]:
# --- Step 5: Print Outputs ---
print("\nResponse to Customer: ", result["response"])


Response to Customer:  We're thrilled you loved your experience!  We apologize for the slower shipping and will strive to improve.
